In [ ]:
# MLp with automatic validation set
from keras.models import Sequential
from keras.layers import Dense
import numpy

In [ ]:
# fix seed
seed = 7
numpy.random.seed(seed)

In [ ]:
# load pima indians dataset
dataset = numpy.loadtxt('pima-indians-diabetes.csv', delimiter=',')

In [ ]:
# split int input (X) and output (Y)
X = dataset[:, 0:8]
Y = dataset[:, 8]

In [ ]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the model
model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10)

In [ ]:
# now split the data using sklearn: 67% for training, 33% for testing
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=seed)

In [ ]:
# create model
model2 = Sequential()
model2.add(Dense(12, input_dim=8, activation='relu'))
model2.add(Dense(8, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=150, batch_size=10)

In [ ]:
# finally, use k-fold cross validation
from sklearn.model_selection import StratifiedKFold

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

In [ ]:
cv_scores = []

In [ ]:
for train, test in kfold.split(X, Y):
    # create model
    kf_model = Sequential()
    kf_model.add(Dense(12, input_dim=8, activation='relu'))
    kf_model.add(Dense(8, activation='relu'))
    kf_model.add(Dense(1, activation='sigmoid'))
    kf_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    kf_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=150, batch_size=10)
    scores = kf_model.evaluate(X[test], Y[test], verbose=0)
    print('%s: %.2f%%' % (model.metrics_names[1], scores[1] * 100))
    cv_scores.append(scores[1] * 100)

print('%.2f%% (+/- %.2f%%)' % (numpy.mean(cv_scores), numpy.std(cv_scores)))

In [ ]:
# do it faster
from deco import concurrent, synchronized

cv_scores2 = []
    
@concurrent(processes=4)
def trainCycle(train, test):
    # create model
    print('creating model')
    kf_model = Sequential()
    kf_model.add(Dense(12, input_dim=8, activation='relu'))
    kf_model.add(Dense(8, activation='relu'))
    kf_model.add(Dense(1, activation='sigmoid'))
    kf_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    kf_model.fit(X[train], Y[train], validation_data=(X[test], Y[test]), epochs=150, batch_size=10)
    scores = kf_model.evaluate(X[test], Y[test], verbose=0)
    print('%s: %.2f%%' % (model.metrics_names[1], scores[1] * 100))
    cv_scores2.append(scores[1] * 100)

def trainAll():
    for train, test in kfold.split(X, Y):
        print('training fold')
        trainCycle(train, test)
    trainCycle.wait()
    print('done')
    return

trainAll()
print('%.2f%% (+/- %.2f%%)' % (numpy.mean(cv_scores2), numpy.std(cv_scores2)))